# Ceneo Scraper

# Biblioteki

In [ ]:
import requests
from bs4 import BeautifulSoup

# Wysyłanie do serwera żądania dostepu do zasobu

In [ ]:
product_id = "94103927"
url = f"https://www.ceneo.pl/{product_id}#tab=reviews"
response = requests.get(url)
response.status_code 

In [ ]:
page_dom = BeautifulSoup(response.text,"html.parser")
print(type(page_dom))
opinions = page_dom.select("div.js_product-review")
print(type(opinions))
opinion = page_dom.select_one("div.js_product-review")
print(type(opinion))
print(opinion)

##Ekstracja składowych pojedynczej opinii

|Składowa|Selektor|Zmienna|
|--------|--------|-------|
|id opinii|["data-entry-id"]|opinion_id|
|autor|span.user-post_author-name|author|
|rekomendacja|span.user-post_author-recomendation > em|recommendation|
|gwiazdki|span.user-post_score-count|rating|
|treść|div.user-post_text|content|
|lista zalet|div.review_feature_title-positives ~ div.review-feature_item|pros|
|lista wad|div.review_feature_title-negatives ~ div.review-feature_item|cons|
|dla ilu przydatna|button.vote-yes > span|useful|
|dla ilu nieprzydatna|button.vote-no > span|useless|
|data wystawienia|span.user-post_published > time:nth-child(1)["datetime"]|publish_date|
|data zakupu|span.user-post_published > time:nth-child(2)["datetime"]|purchase_date|


In [ ]:
opinion_id = opinion["data-entry-id"]        #dorobic to - z select albo select one w zaleznosci czy lista
author = opinion.select_one["user-post_author-name"]
recommendation = opinion["recommended"]
rating = opinion["user-post_score"]
content = opinion.select_one("")
pros = opinion.select_one("")
cons = opinion.select_one("")
useful = opinion.select_one("")
useless = opinion.select_one("")
publish_date = opinion["user-post_published"]
purchase_date = opinion.select_one("")
